# Tutorial 2: Understanding and Using CycleReviewer

## Introduction

CycleReviewer (also known as WhizReviewer) is a set of specialized large language models that have undergone extensive supervised training specifically for academic peer review. The model is available in three sizes:

- 8B parameters (based on Llama3.1)
- 70B parameters (based on Llama3.1)
- 123B parameters (based on Mistral-Large-2)

These models are designed to simulate the peer review process by evaluating research papers across multiple dimensions, providing detailed feedback, and generating comprehensive reviews that closely mirror those produced by human academic reviewers.

## Setting Up the Environment

Let's start by importing the necessary libraries and initializing the CycleReviewer model:

In [1]:
import json

from ai_researcher import CycleReviewer

# Initialize CycleReviewer with the 8B parameter model (for faster inference)
# You can choose "8B", "70B", or "123B" depending on available computational resources
reviewer = CycleReviewer(model_size="8B")

D:\Mike\PycharmProjects\Researcher\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 11-16 19:50:32 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 11-16 19:50:37 [_custom_ops.py:20] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


OSError: WestlakeNLP/CycleReviewer-ML-Llama3.1-8B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

## Loading a Paper to Review

Let's load a paper from a JSON file and prepare it for review:

In [ ]:
# Load a paper from our JSON file
with open('generated_paper.json', 'r', encoding='utf-8') as f:
    papers = json.load(f)

# Print basic information about the paper
print(f"Paper Title: {papers[0]['title']}")
print(f"Abstract length: {len(papers[0]['abstract'])} characters")
print(f"LaTeX content length: {len(papers[0]['latex'])} characters")

## Reviewing a Single Paper

Now, let's use CycleReviewer to evaluate the paper:

In [ ]:
# Generate the review
review_result = reviewer.evaluate([paper['latex'] for paper in papers])

## Parsing and Analyzing Review Results

Let's parse the review to extract structured information including ratings, strengths, weaknesses, and recommendations:

In [ ]:
# Parse the review
review_data = review_result[0]

for i in range(len(review_result)):
    if review_result[i] != None:
        review_data = review_result[i]
        # Print summary of the review
        print(f"Title: {papers[i]['title']}")
        print(f"Average Rating: {review_data['avg_rating']:.2f}/10")
        print(f"Decision: {review_data['paper_decision']}")
        print(f"Number of Reviewers: {len(review_data['reviews'])}")
        print("\nIndividual Ratings:")
        for i, rating in enumerate(review_data['rating']):
            print(f"  Reviewer {i+1}: {rating}")
        print('---***---')

## Let's examine some key feedback from the reviewers:

In [ ]:
# Display key strengths mentioned by reviewers
print("KEY STRENGTHS IDENTIFIED:")
print("-" * 50)
for i, strength in enumerate(review_data['strength']):
    print(f"Reviewer {i+1}:")
    # Print the first 200 characters of each strength for brevity
    print(strength[:500] + "..." if len(strength) > 200 else strength)
    print()

# Display key weaknesses mentioned by reviewers
print("\nKEY WEAKNESSES IDENTIFIED:")
print("-" * 50)
for i, weakness in enumerate(review_data['weaknesses']):
    print(f"Reviewer {i+1}:")
    # Print the first 200 characters of each weakness for brevity
    print(weakness[:500] + "..." if len(weakness) > 200 else weakness)
    print()

### Meta Review

Let's look at the meta review, which synthesizes the individual reviewer feedback:

In [ ]:
print("META REVIEW:")
print("=" * 50)
print(review_data['meta_review'])

## Finding the Best Paper

Now, let's find the highest-rated paper from our reviews:

In [ ]:
# Find the best paper
rating_max = 0
best_paper_num = 0
for i in range(len(review_result)):
    if review_result[i] != None:
        ratings = review_data['avg_rating']
        if ratings > rating_max:
            best_paper_num=i
            rating_max = ratings


print("=" * 50)
print("BEST PAPER SELECTED:")
print(f"Title: {papers[best_paper_num]['title']}...")
print(f"Average Rating: {review_result[best_paper_num]['avg_rating']:.2f}/10")
print(f"Decision: {review_result[best_paper_num]['paper_decision']}")
print("=" * 50)

# Print key strengths from the highest-rated paper's review
best_review = review_result[best_paper_num]
print("\nKEY STRENGTHS OF THE BEST PAPER:")
for i, strength in enumerate(best_review.get('strength', [])):
    if strength:
        print(f"\nStrength {i+1}:")
        print(strength[:300] + "..." if len(strength) > 300 else strength)

## Conclusion

In this tutorial, we've explored how to use CycleReviewer to evaluate academic research papers. We've seen how the model can:

1. Generate detailed reviews that assess papers on multiple dimensions
2. Provide specific feedback on strengths and weaknesses
3. Assign numerical ratings to quantify paper quality
4. Generate meta-reviews that synthesize multiple reviewer perspectives
5. Make accept/reject recommendations

CycleReviewer represents a significant advancement in automating the peer review process, offering researchers, publishers, and educators a powerful tool for evaluating scientific work. While it can provide valuable feedback and insights, it's important to remember that these automated reviews are best used in conjunction with human expert evaluation, especially for final publication decisions.